In [20]:


MODEL = "llama2"

In [21]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

model.invoke("Tell me a joke")

"\nWhy don't scientists trust atoms? Because they make up everything! 😂"

In [22]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser 
chain.invoke("Tell me a joke")

"Sure! Here's one:\n\nWhy don't scientists trust atoms?\nBecause they make up everything!\n\nI hope you found that amusing. If you want to hear another, just let me know!"

In [23]:
from langchain.prompts import PromptTemplate

template = """
Answer the question based on the medicines below. If you can't 
answer the question, reply "I don't know". Don't add reference to it.

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
prompt.format(context="About medicines", question="tell about clavam 650")

'\nAnswer the question based on the medicines below. If you can\'t \nanswer the question, reply "I don\'t know". Don\'t add reference to it.\n\nContext: About medicines\n\nQuestion: Here is a question\n'

In [30]:
ques = input("whats your question?")

In [24]:
chain = prompt | model | parser

response = chain.invoke({"context": "Give the details about given medicine", "question": f"{ques}"})

' Sure, I\'d be happy to help! Based on the information provided, my answer to the question "What\'s your name?" would be "Santiago."'

In [25]:
from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("C:\hackX\druglist.pdf")
pages = loader.load_and_split()
pages[0]

Document(metadata={'source': 'C:\\hackX\\druglist.pdf', 'page': 0}, page_content='Sl No Drug name Category\n1 Lignocaine HCl injection I.P 1% Anaesthetics\n2 Lignocaine +nifedipine ointment Anaesthetics\n3 Prilocaine lignocaine       inj Anaesthetics\n4 Alprazolam tab I.P 0.25 mg,                                                                                               Preanaesthetic medications\n5 Alprazolam tab I.P 0.5 mg\n6 Amitriptyline tab I.P 10 mg Preanaesthetic medications\n7 Midazolam   inj 5 mg/ml Preanaesthetic medications\n8 Midazolam  I nj 5 mg/5 ml Preanaesthetic medications\n9 Midazolam nasal spray 5mg/ml Preanaesthetic medications\n10 Propantheline tab 15 mg Preanaesthetic medications\n11 Paracetamol suppository 100 mg NSAIDS\n12 Paracetamol inj 500 mg NSAIDS\n13 Paracetamol tab 650 mg NSAIDS\n14 Paracetamol tab 1000 mg NSAIDS\n15 Paracetamol drops 100 mg/ml NSAIDS\n16 Diclofenac sodium spray NSAIDS\n17Diclofenac sodium +  Serratiopeptidase Tab (50mg+ \n10mg)NSAIDS\

In [26]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(pages, embedding=embeddings)

In [27]:
retriever = vectorstore.as_retriever()
retriever.invoke("Formoterol+ Budesonide MDI,DPI,Rotacap ")[0]

Document(metadata={'source': 'C:\\hackX\\druglist.pdf', 'page': 14}, page_content='498 Levamisole Immune mediators\n499 Methotrexate inj 50g/2ml ANTICANCER DRUGS\n500 Mycophenolate mofetil Immune mediators')

In [28]:
from operator import itemgetter

chain = (
    {
        "context": itemgetter("question") | retriever,
        "question": itemgetter("question"),
    }
    | prompt
    | model
    | parser
)

In [29]:
questions = [
    "Lignocaine HCl injection I.P 1% "
]

for question in questions:
    print(f"Question: {question}")
    print(f"Answer: {chain.invoke({'question': question})}")
    print()

Question: Lignocaine HCl injection I.P 1% 
Answer: The information you provided is a list of drugs and medical supplies, including lignocaine HCl injection I.P 1%. Here is the information for lignocaine HCl injection I.P 1%:

* Name: Lignocaine HCl injection I.P 1%
* Formulation: Injection, solution
* Strength: 1% (lignocaine)
* Indications: Topical anesthesia for local infiltration, nerve blocks, and epidural anesthesia
* Dosage and administration: The recommended dose is 2-5 mL of the solution for each injection. It should be administered slowly and carefully to avoid inadvertent intravascular injection.
* Contraindications: Known hypersensitivity to lignocaine or any other ingredient in the formulation, as well as in patients with a history of cardiovascular disease, heart block, or respiratory disease.
* Special precautions: The solution should be administered slowly and carefully to avoid inadvertent intravascular injection. In addition, lignocaine can cause a drop in blood pressu